# 3D production of skymaps


### Objective

Creating images of the SNR RX J1713.7-3946.

### Steps

- Create a `~gammapy.data.DataStore` poiting to the relevant data 
- Apply an observation selection to produce a list of observations, a `~gammapy.data.Observations` object.
- Define a geometry of the Map we want to produce, with a sky projection and an energy range.
    - Create a `~gammapy.maps.MapAxis` for the energy
    - Create a `~gammapy.maps.WcsGeom` for the geometry
- Create the necessary makers : 
    - the map dataset maker : `~gammapy.makers.MapDatasetMaker`
    - the background normalization maker, here a `~gammapy.makers.FoVBackgroundMaker`
    - and usually the safe range maker : `~gammapy.makers.SafeRangeMaker`
- Perform the data reduction loop. And for every observation:
    - Apply the makers sequentially to produce the current `~gammapy.datasets.MapDataset`
    - Stack it on the target one.
- Apply a `~gammapy.estimators.ExcessMapEstimator` to compute excess, significance and flux maps of the region.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion

## Defining the datastore and selecting observations

We first use the `~gammapy.data.DataStore` object to access the observations we want to analyse. Here the H.E.S.S. DL3 DR1. 

We can now define an observation filter to select only the relevant observations. 

We can now retrieve the relevant observations

## Exploring the data contained in an observation

In gammapy, you can have a view of the content of an object with the `peek()` method. You can use it on a `Observation`.
It will show synthetic views of the IRFs.

### Exploring event lists


You can access directly a number of informations:

### Accessing and exploring IRFs
You can also access all IRFs stored on the `Observation`.

## Preparing reduced datasets geometry
We define a reference geometry for our analysis, We choose a WCS based geometry with a binsize of 0.02 deg and also define an energy axis: 

Now we can define the target dataset with this geometry.

## Data reduction

### Create the maker classes to be used

The `~gammapy.datasets.MapDatasetMaker` object is initialized as well as the `~gammapy.makers.SafeMaskMaker` that carries here a maximum offset selection.

The `FoVBackgroundMaker` performs the background normalisation outside the exclusion region

### Perform the data reduction loop

### Inspect the reduced dataset

We can use the peek() method to have a brief view of the dataset content.

The dataset contains also maps of projected IRFs. See e.g. the `PSFMap`.

Save the dataset to disc using `~gammapy.datasets.MapDataset.write()` method:

At this stage, we have created a DL4 `MapDataset`. 

## Compute an excess and a significance map

Note that the estimator also build flux maps that can accessed directly. 

### Excess maps in different energy bands

We can also apply the estimator in different energy bands to look for possible differences in the emission as a function of energy.
To do so, we pass `energy_edges` as an argument.
The estimator will find the energy bin closest to the required edge.

# Fit a model to the dataset

Now, we define a global ~gammapy.modeling.model.FoVBackgroundModel in order to finely adjust the level of residual CR backgroud. This should not be forgotten.

Now we assign these models to our reduced dataset:

## Fit the model

The `~gammapy.modeling.Fit` class is orchestrating the fit, connecting the `stats` method of the dataset to the minimizer. By default, it uses `iminuit`.

Its constructor takes a list of dataset as argument.

Now rerun the estimator on the fitted dataset

# Exercises

- Modify the data reduction loop to fit also the background model `tilt` parameter, in addition to the default `norm` fit
- By default the model fit is performed in the full energy range defined by the dataset `mask_safe`. Try to restrict it by using the `Dataset.mask_fit` property
- Repeat the flux points estimation by reoptimizing, in each energy bin, all the model free parameters. This will take longer, but it will provide a more reliable estimation of the source flux in each energy bin
- What is the TS-based significance of RX J1713? You can estimate it by comparing the model likelihood without (null hypothesis) and with the source model 